<a href="https://www.kaggle.com/code/guitaristboy/transformer-from-scratch?scriptVersionId=209085734" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/input-txt/input.txt


In [3]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [4]:
# hyperparameter
# how many independent sequences will we process in parallel?
batch_size = 64
# what is the maximum context ellgnth for predeictions?
block_size = 256 
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = "cuda" if torch.cuda.is_available() else "cpu"
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2

In [5]:
device

'cuda'

In [6]:
torch.manual_seed(1337)

In [7]:
with open('/kaggle/input/input-txt/input.txt','r',encoding='utf-8') as  f:
    text = f.read()

In [8]:
print('Length of the dataset in character is ' , len(text))

Length of the dataset in character is  1115394


In [9]:
# here are the unique character that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [10]:
# create a mapping from characters to integers
stoi = {ch:i for i , ch in enumerate(chars)}
itos = {i:ch for i , ch in enumerate(chars)}
# encoder: take string , output a list of integers
encode = lambda s : [stoi[c] for c in s] 
# decoder: takes input of integers , outputs string
decode = lambda l : ''.join([itos[i] for i in l])
print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [12]:
data = torch.tensor(encode(text),dtype = torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [13]:
# lets now split up the data into train and validation sets
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [14]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
print(f"when input is {context} the target: {target}")

when input is tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
        47, 59, 57,  1, 47

In [26]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split =="train" else val_data
    ix = torch.randint(len(data) - block_size , (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])

    return x, y

xb , yb = get_batch('train')
print('inputs :')
print(xb.shape)
print(xb)
print('targets : ')

print(yb.shape)
print(yb)


print('-------------')

# batch dimension
for b in range(batch_size):
    # time dimension
    for t in range(block_size):
        context = xb[b,:t+1]
        target = yb[b,t]
print(f"When input is {context.tolist()} the target: {target}")

inputs :
torch.Size([64, 256])
tensor([[53, 47, 52,  ..., 59, 58,  0],
        [44, 43, 50,  ..., 53, 56,  1],
        [51, 43,  1,  ..., 54, 56, 53],
        ...,
        [58, 53, 52,  ..., 12,  0, 34],
        [ 1, 44, 39,  ..., 53, 52, 50],
        [39, 60, 43,  ...,  6,  0, 32]])
targets : 
torch.Size([64, 256])
tensor([[47, 52, 58,  ..., 58,  0, 27],
        [43, 50, 50,  ..., 56,  1, 47],
        [43,  1, 43,  ..., 56, 53, 59],
        ...,
        [53, 52,  6,  ...,  0, 34, 47],
        [44, 39, 47,  ..., 52, 50, 63],
        [60, 43, 52,  ...,  0, 32, 46]])
-------------
When input is [39, 60, 43, 52, 57, 6, 1, 58, 39, 49, 47, 52, 45, 1, 39, 52, 45, 56, 63, 1, 52, 53, 58, 43, 6, 0, 20, 39, 60, 43, 1, 50, 43, 44, 58, 1, 51, 43, 1, 47, 57, 57, 59, 43, 50, 43, 57, 57, 11, 1, 39, 52, 42, 1, 63, 53, 59, 56, 1, 44, 39, 58, 46, 43, 56, 5, 57, 1, 40, 50, 43, 57, 58, 6, 0, 13, 57, 1, 46, 43, 1, 44, 56, 53, 51, 1, 46, 43, 39, 60, 43, 52, 1, 51, 43, 56, 47, 58, 57, 1, 47, 58, 6, 1, 61, 47

In [38]:
@torch.no_grad()
def estimate_loss():
    model.eval()  
    out = {}
    with torch.no_grad():
        for split in ['train', 'eval']:
            losses = torch.zeros(eval_iters)
            for k in range(eval_iters):
                X, Y = get_batch(split)
                X, Y = X.to(device), Y.to(device)  
                logits, loss = model(X, Y)
                losses[k] = loss.item()
            out[split] = losses.mean()
    model.train()  
    return out

In [39]:
class Head(nn.Module):
    """ One head of self-attention"""

    def __init__(self,head_size):
        super().__init__()
        self.key = nn.Linear(n_embd,head_size,bias=False)
        self.query = nn.Linear(n_embd,head_size,bias=False)
        self.value = nn.Linear(n_embd,head_size,bias=False)
        self.register_buffer('tril',torch.tril(torch.ones(block_size,block_size)))

        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x) #(B,T,C)
        q = self.query(x) #(B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B,T,C) @ (B, C, T) --> (B,T,T)
        wei = wei.masked_fill(self.tril[:T,:T] == 0 , float('-inf')) #(B,T,T)
        wei = F.softmax(wei , dim = -1) # (B,T,T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) #(B,T,C)
        out = wei @ v #(B,T,T) @ (B,T,C) -> (B,T,C)
        return out

In [40]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_head , head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_head)])
        self.proj = nn.Linear(n_embd,n_embd)
        self.dropout = nn.Dropout(dropout)
    def forward(self,x):
        out = torch.cat([h(x) for h in self.heads], dim = -1)
        out = self.proj(out)
        out = self.dropout(out)
        return out

In [41]:
class FeedForward(nn.Module):
    """ a simple linear layer followed  by a non-linearity """

    def __init__(self,n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd,4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd,n_embd),
            nn.Dropout(dropout)
        )

    def forward(self,x):
        return self.net(x)

In [42]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """
    
    def __init__(self,n_embd , n_head):
        # n_embd: embedding dimension , n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head,head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    def forward(self,x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [43]:
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size,n_embd)
        self.postion_embedding_table = nn.Embedding(block_size , n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd,n_head=n_head) for _ in range(n_layer)])
        # self.sa_heads = MultiHeadAttention(4,n_embd//4) # i.e. 4 heads of 8-dimensional self-attention
        # self.ffwd = FeedForward(n_embd)
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd,vocab_size)
        
    def forward(self, idx , targets=None):
        B , T = idx.shape
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) #(B , T , C)
        pos_emb = self.postion_embedding_table(torch.arange(T,device=device)) #(T,C)
        x = tok_emb + pos_emb # (B , T , C)
        # x = self.sa_heads(x) # applying multiple head of self-attention. (B,T,C)
        # x = self.ffwd(x) # (B,T,C)
        x = self.blocks(x) # (B,T,C) 
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) #(B , T , vocab_size)
        
        if targets is None:
            loss = None
        else:
            B , T , C = logits.shape
            logits = logits.view(B * T , C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits,targets)
        return logits , loss

    def generate(self,idx,max_new_tokens):
        # ids is (B,T) array of indices in the current context
        for _ in  range(max_new_tokens):
            #crop idx to the last block_size tokens
            idx_cond = idx[:,-block_size:]
            # get the predictions
            logits , loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:,-1,:] # becomes(B,C)
            # apply softmax to get probabilites
            probs = F.softmax(logits,dim =-1) # (B,C)
            # sample from the distribution
            idx_next = torch.multinomial(probs,num_samples=1) #(B,1)
            # append sampled index to the running sequence
            idx = torch.cat((idx,idx_next), dim = 1) # (B , T+1)
        return idx

In [44]:
model = BigramLanguageModel()
m = model.to(device)
# logits , loss = m(xb,yb)
# print(logits.shape)
# print(loss)

In [45]:
# # create a PyTorch optimizer
# optimizer = torch.optim.AdamW(m.parameters(),lr = 1e-3)

# for steps in range(10000):

#     # sample a batch of data
#     xb , yb = get_batch("train")

#     # evaluate the loss
#     logits , loss = m(xb,yb)
#     optimizer.zero_grad(set_to_none=True)
#     loss.backward()
#     optimizer.step()

# print(loss.item())

In [46]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(),lr = learning_rate)

In [48]:
for iter in range(max_iters):
    # Evaluate loss every eval_interval
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['eval']:.4f}")

    # Sample a batch of data
    xb, yb = get_batch('train')
    xb, yb = xb.to(device), yb.to(device)  # Move tensors to the correct device

    # Forward pass
    logits, loss = m(xb, yb)

    # Backpropagation
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# Generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)


step 0: train loss 4.3614, val loss 4.3608
step 500: train loss 2.0055, val loss 2.0881
step 1000: train loss 1.6189, val loss 1.7870
step 1500: train loss 1.4523, val loss 1.6576
step 2000: train loss 1.3586, val loss 1.5800
step 2500: train loss 1.2896, val loss 1.5236
step 3000: train loss 1.2403, val loss 1.5145
step 3500: train loss 1.1950, val loss 1.4897
step 4000: train loss 1.1573, val loss 1.4845
step 4500: train loss 1.1212, val loss 1.4831


In [52]:
print(decode(m.generate(torch.zeros((1,1),dtype=torch.long,device=device),max_new_tokens=500)[0].tolist()))


Whilst thou be thy charm?

KING RICHARD III:
Now, princely a wile thou hast obey'd thy crey?
O, when to cure our crown in content? would
You fly dagger? what goes, why every time she zaunts
For our poison, and professes wants to apprower
Shirt-house of this! I think, he crows myself.

GREY:
Pinace from me you will, not that, good farewell,
I have received moies to-day.

KING RICHARD II:
Say, Sir Jot, the Vaughret's, brother build,
Dainours so he knees he place wep his virge.

MENENIUS:
Be thus, 


In [51]:
print(sum(p.numel() for p in m.parameters())/1e6,'M Paramters')

10.788929 M Paramters
